In [ ]:
import pandas as pd 
import numpy as np
from datetime import datetime
import math
from datetime import datetime

# 데이터 읽어오기 
movies=pd.read_csv("/content/drive/MyDrive/영화추천해보기/movies.csv")
ratings=pd.read_csv("/content/drive/MyDrive/영화추천해보기/ratings.csv")

In [ ]:
def sim_cosine(data, name1, name2):
  sum_name1 = 0
  sum_name2 = 0
  sum_name1_name2 = 0
  count = 0
  for movies in data[name1]:
    if movies in data[name2]: # 같은 영화를 봤다면
      sum_name1 += pow(data[name1][movies], 2)
      sum_name2 += pow(data[name2][movies], 2)
      sum_name1_name2 += data[name1][movies]*data[name2][movies]
  # print('sum_name1',sum_name1)
  # print('sum_name2',sum_name2)
  # print('sum_name1_name2',sum_name1_name2)
  return sum_name1_name2 / ( (math.sqrt(sum_name1)*math.sqrt(sum_name2) + 0.00001 ))

def top_match(data, name, index=3, sim_function=sim_cosine):
    li=[]
    for i in data: # 데이터프레임을 돌고
        if name != i : # 자기 자신이 아닐 때만
            li.append( (sim_function(data, name, i), i))
    li.sort() # 오름차순
    li.reverse() # 내림차순
    return li[:index]

def getRecommendation(data, person, k=3, sim_function=sim_cosine):
    result = top_match(data, person, k)
    score = 0 # 평점 합을 위한 변수
    li = []  # 리턴을 위한 리스트
    score_dic = {}  # 유사도 총합을 위한 dict
    sim_dic = {}  # 평점 총합을 위한 dic

    for sim, name in result: # 튜플
        print(sim, name)
        if sim < 0: # 유사도가 양수인 사람만
            for movie in data[name]:
                if movie not in data[person] : # name이 평가를 내리지 않은 영화
                    score += sim * data[name][movie]  # 그 사람의 영화평점 * 유사도
                    score_dic.setdefault(movie, 0)  # 기본값 설정
                    score_dic[movie] += score  # 합계 구함

                    # 조건에 맞는 사람의 유사도와 누적합을 구한다
                    sim_dic.setdefault(movie, 0)
                    sim_dic[movie] += sim

                score = 0  # 영화가 바뀌었으니 초기화
    
    for key in score_dic :
        score_dic[key] = score_dic[key] / sim_dic[key] # 평점 총합 / 유사도 총합
        li.append( ( score_dic[key], key ))  # list[ (tuple) ] 의 리턴을 위하여
    li.sort()
    li.reverse()
    return li

In [ ]:
# 아이템 기반 협업 필터링  
data=pd.merge(ratings,movies,on="movieId")
column=['userId','movieId','rating','title','genres']
data=data[column]
data

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...
1048570,7066,88572,1.5,Fred: The Movie (2010),Comedy
1048571,7066,112412,4.5,Perfect Sisters (2014),Drama|Mystery
1048572,7077,32013,3.5,Loop the Loop (Up and Down) (Horem pádem) (2004),Comedy
1048573,7086,102596,4.5,180° South (180 Degrees South) (180° South: Co...,Documentary|Drama


In [ ]:
user_ratings_expand = {}
a = datetime.now()
for idx in data.index:
  user_id = data.loc[idx,'userId']
  movie_id = data.loc[idx,'movieId']
  score = data.loc[idx, 'rating']
  if user_ratings_expand.get(user_id):
      user_ratings_expand[user_id][movie_id]=score
  else:
      user_ratings_expand[user_id]={movie_id:score}   
b = datetime.now()
print("dict로 바꾸는 시간",b-a)

dict로 바꾸는 시간 0:00:55.071577


In [ ]:
def user_based_recommendation(user_id):
    # # 추천알고리즘
    a = datetime.now()
    getRecommendation(user_ratings_expand, user_id)
    b = datetime.now()
    print('추천돌리는시간', b-a)

user_based_recommendation(1)

0.9999999065420646 6310
0.9999998750000156 6564
0.9999998750000156 2375
추천돌리는시간 0:00:00.487012


In [ ]:
n = 0
for key in user_ratings_expand:
  user_based_recommendation(key)
  n += 1
  if n == 20:
    break

0.9999999065420646 6310
0.9999998750000156 6564
0.9999998750000156 2375
추천돌리는시간 0:00:00.442929
0.9999999397590396 4970
0.9999999200000063 1083
0.9999999166666736 10
추천돌리는시간 0:00:00.281131
0.9999999333333378 1503
0.99999992424243 5879
0.9999999000000099 7086
추천돌리는시간 0:00:00.281164
0.9999999354838751 952
0.9999998901099022 6797
0.9999998901099022 398
추천돌리는시간 0:00:00.478216
0.9999998780487952 3305
0.9999998780487952 2629
0.9999998630137176 6487
추천돌리는시간 0:00:00.346234
0.999999945054948 3156
0.9999998333333611 5109
0.9999997777778269 876
추천돌리는시간 0:00:01.422842
0.9999998518518736 1487
0.9999998095238456 3620
0.9999997777778271 5644
추천돌리는시간 0:00:00.445847
0.9999998888889012 5096
0.99999960000016 3519
0.9999995555557532 434
추천돌리는시간 0:00:01.499629
0.9999995000002501 4753
0.9999988571441633 3119
0.9999980000040001 6507
추천돌리는시간 0:00:01.935681
0.99999992424243 1982
0.9999998809523951 5313
0.9999998780487952 3087
추천돌리는시간 0:00:00.158833
0.9999998901099022 3634
0.9999998780487952 3698
0.9999998666666

In [ ]:
item_ratings_expand = {}
a = datetime.now()
for idx in data.index:
  user_id = data.loc[idx,'userId']
  movie_id = data.loc[idx,'movieId']
  score = data.loc[idx, 'rating']
  if item_ratings_expand.get(user_id):
      item_ratings_expand[user_id][movie_id]=score
  else:
      item_ratings_expand[user_id]={movie_id:score}   
b = datetime.now()
print("dict로 바꾸는 시간",b-a)

In [ ]:
def item_based_recommendation(movie_id):
    # 추천알고리즘
    a = datetime.now()
    getRecommendation(item_ratings_expand, movie_id)
    b = datetime.now()
    print(b-a)

item_based_recommendation(460465)

In [ ]:
moviedata=data.pivot_table(index="movieId",  columns='userId')['rating']
moviedata

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,7081,7082,7083,7084,7085,7086,7087,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.0,NaN,NaN,5.0,NaN,4.0,NaN,4.0,4.5,4.0,4.0,4.5,NaN,3.0,NaN,NaN,5.0,NaN,NaN,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,...,5.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,4.5
2,3.5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,NaN,4.0,NaN,NaN,NaN,3.0,3.0,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
moviedata.columns[0]

numpy.int64

In [ ]:
moviedata[3]

movieId
1         4.0
2         NaN
3         NaN
4         NaN
5         NaN
         ... 
130073    NaN
130219    NaN
130462    NaN
130490    NaN
130642    NaN
Name: 3, Length: 14026, dtype: float64

In [ ]:
#NaN값을 -1로 변경 ( 평점을 계산할 때 양수값만 처리하면 됌)
moviedata.fillna(-1, inplace=True)
moviedata

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,7081,7082,7083,7084,7085,7086,7087,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,4.0,-1.0,-1.0,5.0,-1.0,4.0,-1.0,4.0,4.5,4.0,4.0,4.5,-1.0,3.0,-1.0,-1.0,5.0,-1.0,-1.0,3.0,4.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,5.0,-1.0,...,5.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,-1.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,-1.0,-1.0,4.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,4.0,-1.0,5.0,4.5
2,3.5,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.5,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0
3,-1.0,4.0,-1.0,-1.0,-1.0,3.0,3.0,5.0,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,3.0,-1.0,3.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,3.5,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130073,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
130219,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
130462,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
#kdd 유사도 함수 
from math import sqrt
def sim_distance(data, n1, n2):
    sum=0
    #두 사용자가 모두 본 영화를 기준으로 해야해서 i로 변수 통일(j따로 안 써줌)
    for i in data.loc[n1,data.loc[n1,:]>=0].index:
         if data.loc[n2,i]>=0:
            sum+=pow(data.loc[n1,i]-data.loc[n2,i],2) #누적합 
    return sqrt(1/(sum+1)) #유사도 형식으로 출력 

In [ ]:
# 나와 유사도가 높은 user 매칭 함수
def top_match(data, name, rank = 5, simf = sim_distance):
    simList = []
    for i in data.index[-10:]:
        if name != i:
            simList.append((simf(data, name, i), i))
    simList.sort()
    simList.reverse()    
    return simList[:rank]

In [ ]:
# 추천 시스템 함수
def recommendation(data, person, simf = sim_distance):
    res = top_match(data, person, len(data))
    score_dic = {}
    sim_dic = {}
    myList = []
    for sim, name in res:
        if sim < 0:
            continue
        for movie in data.loc[person, data.loc[person, :] < 0].index:
            simSum = 0
            if data.loc[name, movie] >= 0:
                simSum += sim * data.loc[name, movie]
                
                score_dic.setdefault(movie, 0)
                score_dic[movie] += simSum
                
                sim_dic.setdefault(movie, 0)
                sim_dic[movie] += sim                
    for key in score_dic:
        myList.append((score_dic[key] / sim_dic[key], key))
    myList.sort()
    myList.reverse()
    
    return myList

In [ ]:
# 25번 user가 안본 영화중에서 
#추천 점수가 가장 높은 순으로 예상평점과 영화제목을 추천 (10개까지)
a = datetime.now()
movieList = []
for rate, m_id in recommendation(moviedata, 25):
    movieList.append((rate, movies.loc[movies['movieId'] == m_id, 'title'].values[0]))

print(movieList[:10])

b = datetime.now()

print(b-a)

[(4.5, "Dracula (Bram Stoker's Dracula) (1992)"), (3.5, 'Born Yesterday (1993)'), (3.5, 'Great Muppet Caper, The (1981)'), (3.5, 'Return with Honor (1998)'), (3.0, 'Left Behind II: Tribulation Force (2002)'), (2.5, 'Bully (2001)'), (1.0, "Christmas Vacation (National Lampoon's Christmas Vacation) (1989)"), (0.5, 'Here on Earth (2000)')]
0:00:00.692347


# 피어슨상관계수

In [ ]:
#피어슨 상관계수
def sim_pearson(data, n1, n2): 
    #구현
    sumX=0
    sumY=0
    sumSqX=0 # x 제곱합 
    sumSqY=0 # y 제곱합 
    sumXY=0 #XY 합
    cnt =0 #영화 갯수 
    for i in data.loc[n1,data.loc[n1,:]>=0].index:
        if  data.loc[n2,i]>=0:
            sumX+=data.loc[n1,i]
            sumY+=data.loc[n2,i]
            sumSqX+=pow(data.loc[n1,i],2)
            sumSqY+=pow(data.loc[n2,i],2)
            sumXY+=(data.loc[n1,i])*(data.loc[n2,i])
            cnt+=1
            global num # 전역변수 선언
            global den # 전역변수 선언
            num=sumXY-((sumX*sumY)/cnt)
            den= (sumSqX-(pow(sumX,2)/cnt))*(sumSqY-(pow(sumY,2)/cnt))
    return num/sqrt(den+0.00001) # 분모=0방지

In [ ]:
#나와 유사도가 높은 user와 매칭 함수
def top_match(data, name, rank = 5, simf = sim_pearson):
    simList = []
    for i in data.index:
        if name != i:
            if simf(data, name, i) is not None:
                simList.append((simf(data, name, i), i))
    simList.sort()
    simList.reverse()
    return simList[:rank]

In [ ]:
#추천 시스템 함수
def recommendation(data, person, simf = sim_pearson):
    res = top_match(data, person, len(data))
    score_dic = {}
    sim_dic = {}
    myList = []
    for sim, name in res:
        if sim < 0:
            continue
        for movie in data.loc[person, data.loc[person, :] < 0].index:
            simSum = 0
            if data.loc[name, movie] >= 0:
                simSum += sim * data.loc[name, movie]
                
                score_dic.setdefault(movie, 0)
                score_dic[movie] += simSum
                
                sim_dic.setdefault(movie, 0)
                sim_dic[movie] += sim                
    for key in score_dic:
        myList.append((score_dic[key] / sim_dic[key], key))
    myList.sort()
    myList.reverse()
    
    return myList

In [ ]:
a = datetime.now()
movieList = []
for rate, m_id in recommendation(moviedata, 1):
    movieList.append((rate, movies.loc[movies['movieId'] == m_id, 'title'].values[0]))
    if len(movieList)==10: #10 개되면 멈추기 
        break
print(movieList[:10]) # 10개만 추출 

b = datetime.now()

print(b-a)

[(5.0, 'Siam Sunset (1999)'), (5.0, 'Faust (1926)'), (4.999999999999998, 'Mission to Mars (2000)'), (4.937573538543576, 'Dinosaur (2000)'), (4.936738122276483, '1969 (1988)'), (4.889092433581694, 'Austin Powers: International Man of Mystery (1997)'), (4.885140668693917, 'My Fair Lady (1964)'), (4.86901409423573, 'Zombie Holocaust (a.k.a. Doctor Butcher M.D.) (Zombi Holocaust) (1980)'), (4.842156691080684, 'Torso (1973)'), (4.817939530203945, 'Torn Curtain (1966)')]
0:20:58.731185


# 코사인유사도 추천

In [ ]:
#데이터 불러오기 
import pandas as pd 
data=pd.read_csv("movies_metadata.csv",encoding="UTF-8")
#데이터가 많은 관계로 20000개까지 짜름 
data=data.head(20000)